## Introduction to FALQON

In this demo, we'll be implement the FALQON algorithm, standing for *Feedback-based ALgorithm for Quantum OptimizatioN*, introduced by [Magann, Rudinger, Grace & Sarovar (2021)](https://arxiv.org/pdf/2103.08619.pdf). It is similar in spirit to the [QAOA](https://arxiv.org/pdf/1411.4028.pdf), but scales with resources differently, using iterative feedback steps rather than a global optimization over parameters. We will show how to implement FALQON in PennyLane and test its performance on the **MaxClique** problem in graph theory!

### Theory

To solve combinatorial optimization problems using a quantum computer, a typical strategy is to encode the solution to the problem as the ground state of *cost Hamiltonian* $H_C$, and choose some strategy to drive the system from a known initial state into this ground state. FALQON falls under this broad scheme. The driving strategy is implemented with a *driving Hamiltonian* $H_D$ with a single control parameter $\beta(t)$, so the system evolves according to

$$
i\partial_t |\psi(t)\rangle = (H_C + \beta(t) H_D) |\psi(t)\rangle,
$$

setting $\hbar = 1$. We would ultimately like to minimize the expectation value of $\langle H_C\rangle$, so a reasonable driving strategy is simply to decrease this expectation with time:

$$
\partial_t \langle H_C\rangle_t = \partial_t \langle \psi(t)|H_C|\psi(t)\rangle = i \beta(t)\langle [H_D, H_C] \rangle_t \leq 0,
$$

where we used the product rule and Schrödinger's equation. An easy way to satisfy this equation is to pick $\beta(t) = -\langle i[H_D, H_C] \rangle_t$, so that

$$
\partial_t \langle H_C\rangle_t = -|\langle i[H_D, H_C] \rangle_t|^2 \leq 0.
$$

(Note that we bring the $i$ into the expectation to give a Hermitian operator.)
Using techniques from [control theory](https://arxiv.org/pdf/1304.3997.pdf), it is possible to show this will eventually drive the system into the ground state. But there are two issues: (a) first, we need to somehow *evaluate* the expectation $A(t) := i\langle [H_D, H_C] \rangle_t$ to implement the driving strategy, and (b) we must do so continuously in time! This seems rather impractical. In order to solve the first problem, we can perform a [Trotter-Suzuki](https://en.wikipedia.org/wiki/Lie_product_formula) decomposition of the time evolution operator $|\psi(t)\rangle = U(t)|\psi_0\rangle$:

$$
U(t) \approx U_D(\beta_\ell) U_C U_D(\beta_{\ell-1}) U_C\cdots
U_D(\beta_1) U_C, \quad U_C = e^{-iH_C \Delta t}, \quad U_D(\beta_k) =
e^{-i\beta_k H_D \Delta t},
$$

where $\Delta t = t/2\ell$ and $\beta_k = \beta(2k\Delta t)$. Our discrete-time driving strategy is to use the value of $A(t)$ at the previous time-step:

$$
\beta_{k+1} = -A_k = -A(2k\Delta t).
$$

This leads immediately to the FALQON algorithm. On step $k$, we perform the following three substeps:

1. Prepare the state $|\psi_k\rangle = U_D(\beta_k) U_C \cdots U_D(\beta_1) U_C|\psi_0\rangle$,
2. Measure the expectation value $A_k = \langle i[H_C, H_D]\rangle_k$.
3. Set $\beta_{k+1} = -A_k$.

Provided $\Delta t$ is small enough to ensure negligible error in our Trotter-Suzuki decomposition, after enough steps we will approach the ground state. How small $\Delta t$ needs to be, and how large $\ell$, are fiddly and problem-dependent questions we won't get into now. Is this a good algorithm? One simple measure is the *sampling complexity* $N_S$, the number of times we need to run a circuit (ignoring the resources required by the circuit itself). This will obviously depend on

- the number of samples $s$ required to perform a single measurement of $i[H_C, H_D]$;
- the number of measurements $m$ we want to take in order to approximate the expectation $\langle i[H_C, H_D]\rangle$;  and
- the total number of steps $\ell$.

Multiplying these gives a rough estimate for the sampling complexity $N_s = \mathcal{O}(sm\ell)$. We can contrast this with QAOA, which optimizes $\langle H_C\rangle$ over $2\ell$ parameters (since it modulated the cost Hamiltonian as well). If it uses gradient descent, then each paramter will require separate circuit runs, leading to sampling complexity $N_S = \mathcal{O}(qm\ell)$, where $q$ is the number of runs needed for the classical optimization protocol. This suggests that FALQON is favorable when $s \leq q$, i.e. it takes fewer circuit calls to measure $i[H_C, H_D]$ than it does to classically optimize a single parameter.

### A flightless FALQON in PennyLane

Before launching into a concrete problem, let's describe how to set up the general FALQON algorithm in PennyLane. The code will be illustrative and not executable, but will prepare us for the next section. FALQON shares many structural characteristics with [QAOA](https://pennylane.ai/qml/demos/tutorial_qaoa_intro.html), and in fact, we can use it to perform the subroutines for FALQON! We recommend looking at that tutorial for further background information. Let's start by defining the cost and driving Hamiltonians using PennyLane's `Hamiltonian` operation. We also will typically need to put in the commutator by hand:

In [3]:
import pennylane as qml

cost_H = qml.Hamiltonian(
    # your cost Hamiltonian H_C goes here
)

drive_H = qml.Hamiltonian(
    # your driving Hamiltonian H_D goes here
)

comm_H = qml.Hamiltonian(
    # your commutator i[H_C, H_D] goes here
)

For step $k$ of FALQON, we prepare state $|\psi_\rangle$ by applying $k$ layers, with each layer consisting of a cost step (dependent on $\Delta t$), and then a driving step (dependent on both $\Delta t$ and $\beta$). We can code this using the `cost_layer()` and `mixer_layer()` methods from the QAOA module, which we first import:

In [ ]:
from pennylane import qaoa

def falqon_layer(beta, delta_t):
    qaoa.cost_layer(delta_t, cost_h)
    qaoa.mixer_layer(delta_t*beta, drive_h)

We then apply these layers $k$ times using the `layer()` method, and using the iteratively generated list of $\beta$ values so far:

In [ ]:
def falqon_circuit(step, beta_list, delta_t):
    # initial state preparation
    qml.layer(falqon_layer, step, beta_list, delta_t)

Finally, to determine the next $\beta$ value, we need to measure the commutator $i[H_C, H_D]$. We run a device on the QULACS simulator, which is designed for large parametric circuits, and append the resulting value of $\beta$ to an input list of values given so far:

In [ ]:
dev = qml.device("qulacs.simulator", wires=wires)

def add_beta(step, beta_list, delta_t):
    A = qml.ExpvalCost(circuit(beta_list, delta_t), comm_h, dev)
    return beta_list.append(-A)        

This computes the expectation of $A_k$ exactly, but on a real device we will have to think much more carefully about how to measure the commutator. Finally, we iterate and produce as an output the estimated cost $\langle H_C\rangle_\ell$ and the resulting sequence $\{\beta_k\}$ which generates our state:

In [ ]:
def falqon(ell, delta_t):
    beta_list = [0]
    for step in range(ell):
        beta_list = add_beta(step, beta_list, delta_t)
    min_cost = qml.ExpvalCost(circuit(beta_list, delta_t), cost_h, dev)
    return (min_cost, beta_list)

Let's see how this works on a real example!